In [1]:
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh


# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [2]:
NUM_POINTS = 10


plot = k3d.plot(name='points')
plot.display()

model = Model("./data/Dude.obj")
model.generate_view_points(NUM_POINTS)

observations = []
for view_point_idx in range(NUM_POINTS):
    observation = model.get_observation(view_point_idx)
    
    plot = observation.illustrate(plot, size=0.04)
    plot = illustrate_points([model.get_point(view_point_idx)], size=1.0, plot=plot)
    sleep(2)
    
    observations.append(observation)
    
combined_observation = combine_observations(observations)
reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
print(loss)

illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Output()

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


0.059667802116076925


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…